In [34]:
import graphlab
from utils.em_utilities import *
import gensim
from gensim.parsing.preprocessing import STOPWORDS

In [7]:
wiki = graphlab.SFrame('../Data/people_wiki.gl/')
wiki['tf_idf'] = graphlab.text_analytics.tf_idf(wiki['text'])

In [9]:
tf_idf, map_index_to_word = sframe_to_scipy(wiki, 'tf_idf')

In [10]:
tf_idf = normalize(tf_idf)

In [39]:
def get_words_for_gensim(row):
    words = row.keys()
    words = [w for w in words if w not in STOPWORDS]
    return words

In [40]:
wiki['words'] = wiki['tf_idf'].apply(get_words_for_gensim)

In [41]:
%time id2word_wiki = gensim.corpora.Dictionary(wiki['words'])
print(id2word_wiki)

CPU times: user 44.4 s, sys: 1.09 s, total: 45.5 s
Wall time: 28.6 s
Dictionary(547647 unique tokens: [u'biennials', u'verplank', u'shatzky', u'woode', u'damfunk']...)


In [42]:
# ignore words that appear in less than 20 documents or more than 10% documents
id2word_wiki.filter_extremes(no_below=20, no_above=0.1)
print(id2word_wiki)

Dictionary(26386 unique tokens: [u'biennials', u'theoryhe', u'woods', u'francesco', u'woody']...)


In [45]:
mm_corpus = [id2word_wiki.doc2bow(tokens) for tokens in wiki['words']]

In [47]:
%time tfidf_model = gensim.models.TfidfModel(mm_corpus , id2word=id2word_wiki)

CPU times: user 1.04 s, sys: 8 ms, total: 1.04 s
Wall time: 1.03 s


In [52]:
%time lda_model = gensim.models.LdaModel(tfidf_model[mm_corpus], num_topics=25, id2word=id2word_wiki, passes=4)

CPU times: user 7min 3s, sys: 2.04 s, total: 7min 5s
Wall time: 3min 40s


In [54]:
import pyLDAvis.gensim as gensimvis
import pyLDAvis

In [58]:
vis_data = gensimvis.prepare(lda_model, mm_corpus, id2word_wiki)
pyLDAvis.display(vis_data)